### Exemplo de treinamento simples

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import pickle
from simple_robot import feature_eng
# Baixando os dados de DOGE COIN
df = pd.read_parquet('https://drive.google.com/u/0/uc?id=17c2r9qbnsxPVxaYukrp6vhTY-CQy8WZa&export=download')

#### Calculando o target (y)

In [4]:
# Calculando qual a média de close dos próximos 10min
df['forward_average'] = df[::-1]['close'].rolling(10).mean()[::-1].shift(-1)

# Target será a diferença percentual do 'forward_average' com o 'close' atual 
df['target'] = 100*(df['forward_average'] - df['close']) / df['close']

#df.head(3)
# Outra possibilidade: target como a diferença entre o proximo minuto e o atual: df['diff']= -df['close'].diff(-1)

#### Calculando as features (x)

In [5]:
# Toda a parte de criação de features está no arquivo simple_robot.py. Aqui apenas chamamos a função. Isso é útil, pois conseguimos usar a mesma função no momento de colocar o robô em produção
df = feature_eng(df)
#df.head(5)

KeyError: 'datetime'

#### Separando em treino/ teste
Separando usando data. Isso é importante, pois precisamos entender se os modelos criados em um tempo passado continua sendo útil em um tempo futuro.

In [4]:
test_treshold = '2021-06-01 00:00:00'

train = df[df.index <= test_treshold]
test = df[df.index > test_treshold]

X_train = train.drop(columns=['target'])
y_train = train['target']

X_test = test.drop(columns=['target'])
y_test = test['target']

# Modelo linear simples
model = sm.OLS(y_train,X_train).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     1320.
Date:                Sun, 08 Aug 2021   Prob (F-statistic):               0.00
Time:                        12:54:02   Log-Likelihood:            -7.4021e+05
No. Observations:             1000328   AIC:                         1.480e+06
Df Residuals:                 1000318   BIC:                         1.481e+06
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                2.6935      0.122     22.041      0.000       2.454       2.933
close              -12.4440      0.651    -19.120      0.000     -13.720     -11.168
volume            4.515e-09   8.09e-11     55.808      0.000    4.36e-09    4.67e-09
number_of_trades  3.849e-06    6.4e-07      6.009      0.000    2.59e-06     5.1e-06
lag_1               -0.0266      0.003    -10.584      0.000      -0.031      -0.022
lag_2               -0.0665      0.002    -35.035      0.000      -0.070      -0.063
ma_10               13.1863      0.997     13.225      0.000      11.232      15.141
ma_30               -0.8399      0.617     -1.361      0.174      -2.050       0.370
ratio_ma            -2.6946      0.122    -22.052      0.000      -2.934      -2.455
time              1.798e-10   2.05e-09      0.087      0.930   -3.85e-09    4.21e-09
==============================================================================
Omnibus:                   483216.165   Durbin-Watson:                   0.279
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        690137035.388
Skew:                           0.746   Prob(JB):                         0.00
Kurtosis:                     131.669   Cond. No.                     2.06e+10
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.06e+10. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### Resultado do Modelo Linear

In [6]:
y_hat = model.predict(X_test)
MSE = ((y_hat - y_test)**2).mean()
MSE

0.17070794256499783

In [7]:
MAE = ((y_hat - y_test).abs()).mean()
MAE

0.26346826055111083

#### Referência
É sempre recomendado ter valores de referência, para saber se seu modelo é ou não melhor do que outras alternativas "naive"

Abaixo, um exemplo de resultado Naive, assumindo todos 0

In [8]:
MSE_assuming_all_zero = (y_test**2).mean()
MSE_assuming_all_zero

0.16986135624179463

In [9]:
MAE_assuming_all_zero = (y_test.abs()).mean()
MAE_assuming_all_zero

0.2633675092910254

In [10]:
# Salvando o modelo em um arquivo pickle para ser utilizado nas etapas seguintes
filename = 'model_dummy.pickle'
pickle.dump(model, open(filename, 'wb'))